In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import phik 
from phik.report import plot_correlation_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# варианты моделей
from catboost import CatBoostClassifier
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from lightautoml import automl
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
# pyboost

# для ансамблирования 
from sklearn.ensemble import VotingClassifier

# заморозка сидов
import torch
import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1001)

In [ ]:
df = pd.read_csv('train_1.csv', index_col = 0)
df

In [ ]:
df.sort_index(inplace=True)
df

In [ ]:
pred_df = pd.read_csv('test_1.csv', index_col = 0)
pred_df

In [ ]:
pred_df.sort_index(inplace=True)
pred_df

In [ ]:
set(df.columns) - set(pred_df.columns)

In [ ]:
print(df.info())
df

In [ ]:
df.isna().sum().sum()

In [ ]:
pred_df.isna().sum().sum()

In [ ]:
# сравнить показатели датафреймов

In [ ]:
Counter(df.machine_type), Counter(pred_df.machine_type)

In [ ]:
Counter(df['failure_flag'])

In [ ]:
le = LabelEncoder()

df['machine_type'] = le.fit_transform(df['machine_type'])
df

In [ ]:
pred_df['machine_type'] = le.fit_transform(pred_df['machine_type'])

In [ ]:
phik_matrix = df.phik_matrix()

plot_correlation_matrix(phik_matrix.values,
                        x_labels = phik_matrix.columns,
                        y_labels = phik_matrix.index,
                        title = 'phik correlation matrix',
                        fontsize_factor=0.8, figsize=(11, 6)
)

In [ ]:
# удалить колонки с слишком высокой корреляцией

In [ ]:
phik_matrix = pred_df.phik_matrix()

plot_correlation_matrix(phik_matrix.values,
                        x_labels = phik_matrix.columns,
                        y_labels = phik_matrix.index,
                        title = 'phik correlation matrix',
                        fontsize_factor=0.8, figsize=(11, 6)
)

In [ ]:
# сравнить корредяции в датафреймах

In [ ]:
X = df.drop(columns = ['failure_flag'])
X

In [ ]:
y = df.failure_flag
y

In [ ]:
# подобрать метод валидации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
model1 = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, verbose=0)
model2 = CatBoostClassifier(iterations=300, depth=8, learning_rate=0.05, verbose=0)

In [ ]:
model1.fit(X_train, y_train)

In [ ]:
feature_importance = model1.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(15, 10))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Feature Importance')

In [ ]:
# Создание ансамбля с голосованием
model = VotingClassifier(
    estimators=[
        ('cat1', model1), 
        ('cat2', model2)
     ],
    verbose=True,
    # n_jobs = -1,
    voting='hard'  # Можно использовать 'hard' или 'soft'
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
f1_score(y_test, model.predict(X_test), average='macro')

In [ ]:
pred = model.predict(pred_df)
pred

In [ ]:
pred_df['failure_flag'] = pred
pred_df.to_csv('sub.csv', index=False)